# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:

import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,shortname,state
0,Dade,12345,Rick Scott,FL,Florida
1,Broward,40000,Rick Scott,FL,Florida
2,Palm Beach,60000,Rick Scott,FL,Florida
3,Summit,1234,John Kasich,OH,Ohio
4,Cuyahoga,1337,John Kasich,OH,Ohio


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [17]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects.json')
data =json.load((open('data/world_bank_projects.json')))

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [7]:
#1. Fill top 10 countries with most projects
sample_json_df.countryshortname.value_counts().sort_values(ascending = False).head(10)


Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Morocco               12
Bangladesh            12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

In [18]:
#2.Find the top 10 major project themes (using column 'mjtheme_namecode')

themes = json_normalize(data, 'mjtheme_namecode')
distinctRows = themes.drop_duplicates()
CodeNamePair = distinctRows[distinctRows.name!=''].sort('code')

Mjtheme_NameCode_Max10 = themes.groupby('code').count().nlargest(10 , 'name')
Mjtheme_NameCode_Max10.rename(columns={'name':'Name_Count'} , inplace=True)
df_result = pd.merge(CodeNamePair,Mjtheme_NameCode_Max10, left_on ='code' , right_index=True)
df_result.sort('Name_Count' , ascending=False)

C:\Users\rvenkat\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
C:\Users\rvenkat\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,code,name,Name_Count
6,11,Environment and natural resources management,250
18,10,Rural development,216
0,8,Human development,210
5,2,Public sector governance,199
3,6,Social protection and risk management,168
11,4,Financial and private sector development,146
8,7,Social dev/gender/inclusion,130
4,5,Trade and integration,77
53,9,Urban development,50
2,1,Economic management,38


In [21]:
#3 . In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
codeNamePairDict= CodeNamePair.set_index('code').to_dict()
def FillThemeNames(x):
    return codeNamePairDict['name'][x]
themes['name'] = themes['code'].apply(FillThemeNames)